# Deconvolution Tool

## How it works
1. Reads in the spectrum as a .csv and converts to eV
2. Clips the min-max limits of the spectrum
3. Perfomrs any baseline corrections
4. (optional) AutoClips the spectum based on the lowest points either side of the maximum point
5. (optional) Smooths the raw data -> step 11
6. Normalises the spectrum (by area)
7. (optional) Smooths the spectrum before differentiatiating
8. (optional) Differentiates however many number of times
    * (optional) Applies extra smoothing after each step
9. Normalises again in case y it changed during differentiation
10. Uses least squares regression to fit the selected number of Gaussians (or their differentiated equivalent)
    * If the convergence criteria is not met (comparison of residuals) the process is repeated with more Gaussians within the allowed range
11. If th process was performed in derivative space, the locations of the fitted peaks are taken and are re-fitted against the non-derivative space (it's easier to visually inspect)
    * This is the raw (potentially smoothed) spectrum form step 5
12. If you're happy, hit save and a folder will be created at the spectrum readin location with the following files in it:
    * A settings.toml file that has all of the parameters that were chosen
    * CSV files of:
        * List of Gaussians from the derivative fitting
        * List of Gaussians from the non-derivative fitting
        * Raw spectrum (with any applied raw smoothing)
        * Differentiated spectrum (with any applied raw smoothing)
        
## The Smoothing algorithms
### Gaussian kernel
The simpler algorithm here is the Gaussian kernel, which is quite *blurry*, simple and cheap
* Keep the size of this kernel relatively low (max ~4), or you will start losing spectral features

### Savitzky–Golay (savgol)
The more robust implementation is the Savitzky–Golay filter, which fits a window (of your choosing) to a polynomial (with an order of your choosing), and either does so in derivative space or not.
* The window can be quite large (10+), and the polynomial order can be kept relatively low (2-3)
* I've not had much luck with doing this is derivative space, but YMMV
* **Be aware of the edge of your spectrrum** While this filter is better at preserving spectral details, it has a habit of creating weird artefacts at the edge of your spectrum, that will interfere with the least squares fitting procedure
    
#### Here is a pictoral example of the Savitzky–Golay Filter in action
![](https://upload.wikimedia.org/wikipedia/commons/8/89/Lissage_sg3_anim.gif)

### ⬇︎⬇︎⬇︎Set this directory to where you store your data first!!!⬇︎⬇︎⬇︎

In [1]:
##########################
##########################
#### Global Settings #####
##########################
##########################

data_path = '/Users/adrea/gdrive/Monash/PhD/Fluorophore/data'

##########################
##########################

In [ ]:
from deconv import gaussian0FuncList, gaussian1FuncList, gaussian2FuncList, gaussian3FuncList
from deconv import gaussian0FuncList as funcList
from deconv import gaussian0_func, gaussian1_func, gaussian2_func, gaussian3_func
from deconv import Gaussian, evToNm, nmToEv, baseLineCorrect, sortPeaks
from deconv import clipFunc, process, backProcess, smoothing, saveSpectrum
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import HTML, display, clear_output
from ipyfilechooser import FileChooser
from tqdm.notebook import tqdm_notebook

def loadCSV(clip: tuple[float, float]) -> tuple[list[float], list[float]]:
    file = fc.selected

    if file == None:
        raise Exception('File not selected')
        
    with open(file, 'r') as f:
        lines = f.readlines()
        
    x, y = ([], [])
    for line in lines:
        splitline = line.split(',')
        if len(splitline) != 1:
            try:
                x_temp, y_temp = [float(i) for i in splitline[0:2]]
                x += [x_temp]
                y += [y_temp]
            except ValueError:
                pass

    x_ev = nmToEv(x)
    less = np.less(x_ev, clip[1])
    more = np.greater(x_ev, clip[0])
    xor = np.logical_xor(less, more)
    x = np.delete(x, xor)
    y = np.delete(y, xor)
    
    return x, y

def fit(baseLine: str, derivLevel: int, autoClip: bool, clip: tuple[float, float], amp: tuple[float, float], sigma: tuple[int, int], convergence: float,
        gaussRange: tuple[int, int], smoothingRaw: int, preSmoothingSigma: int, smoothingSigma: int, smoothingType: str, savgolPoly: int, savgolDeriv: int, maxIter: int,
        ampCutoff: tuple[float, float], save: bool) -> None:
    clear_output()
    plt.close('all')
    
    x_nm, y = loadCSV(clip)
    baseLineShift = True if baseLine == 'Shift' else False
    baseLineLevel = True if baseLine == 'Level' else False

    if baseLine != 'None':
        y = baseLineCorrect(y, baseLineShift, baseLineLevel)

    if autoClip:
        x_nm, y = clipFunc(x_nm, y)

    x = nmToEv(x_nm)
    
    if smoothingRaw > 0:
        y = smoothing(smoothingType, y, smoothingRaw, savgolPoly, savgolDeriv)
        
    y_norm = np.divide(y, np.linalg.norm(y))
    y_zeroth = y_norm

    if preSmoothingSigma > 0:
        y_norm = smoothing(smoothingType, y_norm, preSmoothingSigma, savgolPoly, savgolDeriv)

    global derivFuncList

    if derivLevel == 0:
        derivFuncList = gaussian0FuncList
        gaussian_func = gaussian0_func
    elif derivLevel == 1:
        derivFuncList = gaussian1FuncList
        gaussian_func = gaussian1_func
    elif derivLevel == 2:
        derivFuncList = gaussian2FuncList
        gaussian_func = gaussian2_func
    elif derivLevel == 3:
        derivFuncList = gaussian3FuncList
        gaussian_func = gaussian3_func
        
    for i in range(derivLevel):
        y_norm = np.gradient(y_norm, x)
        if smoothingSigma > 0:
            y_norm = smoothing(smoothingType, y_norm, smoothingSigma, savgolPoly, savgolDeriv)
    y_norm = np.divide(y_norm, np.linalg.norm(y_norm))

    converged = False
    with tqdm_notebook(total=(gaussRange[1] - gaussRange[0]) + 1) as pBar:
        for ngauss in range(gaussRange[0], gaussRange[1]):
            amps, residual, popt_gauss, pcov_gauss, error = process(x, y_norm, ngauss, amp, sigma, maxIter, derivLevel)
            if error:
                converged = False
                break
            pBar.update(1)
            pBar.set_description(f'Residual: {residual:.6f}')
            conv = 1 * 10**(- convergence)
            if residual <= conv:
                opt_ngauss = ngauss
                converged = True
                break
        pBar.n = gaussRange[1] - gaussRange[0]
        pBar.update()
    if not converged:
        print("Not Converged")
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 6))
        ax1.plot(x, y_norm, 'red', lw=1)
        ax1.axhline(0, c='k', lw=0.5)
        # ax1.plot(x, np.subtract(0,shiftAmount))
        ax1.invert_xaxis()
        ax1.set_ylabel("Absorbance/Emission (AU)")
        ax2.plot(x, y_zeroth, 'red', lw=1)
        ax2.axhline(0, c='k', lw=0.5)
        ax2.invert_xaxis()
        ax2.set_ylabel("Absorbance/Emission (AU)")
        plt.show()
        return

    gauss = []
    gaussObjectList = []
    count = 1
    for i in popt_gauss:
        if count != 3:
            gauss += [i]
            count += 1
        else:
            count = 1
            loc = round(gauss[1], 3)
            nm = int(round(evToNm(loc), 0))

            gaussObjectList += [Gaussian(loc, i, gauss[0])]
            gauss = []
    if derivLevel == 0:
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 6), sharex=True, height_ratios=[3, 1])
        ax2.invert_xaxis()
    else:
        fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(8, 8), sharex=True, height_ratios=[3, 1, 3, 1])
        ax4.invert_xaxis()

    for i in range(0, ngauss * 3, 3):
        ax1.fill_between(x, gaussian_func(x, *popt_gauss[i:i + 3]), alpha=0.3)
    ax1.plot(x, y_norm, 'red', lw=1)
    ax1.plot(x, derivFuncList[ngauss](x, *popt_gauss), 'k--', lw=1)
    ax1.axhline(0, c='k', lw=0.5)
    if derivLevel == 0:
        ax1.set_ylabel("Absorbance/Emission (AU)")
    else:
        ax1.set_ylabel(f"Absorbance/Emission (AU)\nderiv ({derivLevel}) space")
    ax2.set_xlabel("E (eV)")
    ax2.set_ylabel('Residuals')
    ax2.axhline(0, c='k', lw=0.5)
    residualList = np.subtract(derivFuncList[ngauss](x, *popt_gauss), y_norm)
    ax2.scatter(x, residualList, s=0.1)
    if derivLevel != 0:
        ax2.set_xlabel('')
        ax3.plot(x, y_zeroth, 'red', lw=1)
        locList = popt_gauss[1::3]
        ampList = popt_gauss[0::3]

        amps_nonDeriv, residual_nonDeriv, popt_gauss_nonDeriv, pcov_gauss_nonDeriv, culledNgauss = backProcess(x, y_zeroth, amp, sigma, locList, ampList, maxIter, ampCutoff)
        gauss_nonDeriv = []
        gaussObjectList_nonDeriv = []
        count = 1
        for i in popt_gauss_nonDeriv:
            if count != 3:
                gauss_nonDeriv += [i]
                count += 1
            else:
                count = 1
                loc = round(gauss_nonDeriv[1], 3)
                gaussObjectList_nonDeriv += [Gaussian(loc, i, gauss_nonDeriv[0])]
                gauss_nonDeriv = []

        ax3.plot(x, funcList[culledNgauss](x, *popt_gauss_nonDeriv), 'k--', lw=1)
        for i in range(0, culledNgauss * 3, 3):
            ax3.fill_between(x, gaussian0_func(x, *popt_gauss_nonDeriv[i:i + 3]), alpha=0.3)
        ax3.axhline(0, c='k', lw=0.5)
        ax3.set_ylabel("Absorbance/Emission (AU)\nnon-deriv space")
        ax4.set_xlabel("E (eV)")
        ax4.set_ylabel('Residuals')
        ax4.axhline(0, c='k', lw=0.5)
        residualList_nonDeriv = np.subtract(funcList[culledNgauss](x, *popt_gauss_nonDeriv), y_zeroth)
        ax4.scatter(x, residualList_nonDeriv, s=0.1)

    if derivLevel != 0:
        print('#################### Deriv Fit ####################')
    print(f'Residual: {residual:.2e} ≤ {conv:.2e}')
    print(f'Gaussians: {ngauss}')
    if derivLevel != 0:
        for gauss in sortPeaks(gaussObjectList):
            print(f'Gauss - loc: {gauss.center:.3f} eV ({evToNm(gauss.center):.0f} nm) amp: {gauss.amplitude:.5f}  width: {gauss.width:.3f}')
    if derivLevel != 0:
        print('\n##################### Re-Fit #####################')
        print(f'Residual: {residual_nonDeriv:.2e}')
        print(f'Gaussians: {culledNgauss}')
        for gauss in sortPeaks(gaussObjectList_nonDeriv):
            print(f'Gauss - loc: {gauss.center:.3f} eV ({evToNm(gauss.center):.0f} nm) amp: {gauss.amplitude:.5f}  width: {gauss.width:.3f}')
    plt.show()
    if save:
        saveSpectrum(fc.selected_filename,
                     fc.selected_path,
                     baseLine,
                     clip,
                     autoClip,
                     derivLevel,
                     amp,
                     sigma,
                     convergence,
                     gaussRange,
                     maxIter,
                     ampCutoff,
                     smoothingRaw,
                     preSmoothingSigma,
                     smoothingSigma,
                     smoothingType,
                     savgolPoly,
                     savgolDeriv,
                     sortPeaks(gaussObjectList),
                     sortPeaks(gaussObjectList_nonDeriv),
                     x,
                     y_zeroth,
                     y_norm)
    return

fc = FileChooser(data_path)
fc.filter_pattern = '*.csv'

fit_widg = widgets.interactive(fit, {'manual': True, 'manual_name': 'Deconvolute'},
                               clip=widgets.FloatRangeSlider(min=1.54, max=4.13, step=0.001, value=[0, 6.19], description='Spectra Range (Manual Clipping)', orientation='horizontal', readout=True),
                               autoClip=widgets.ToggleButton(value=False, description='Auto Clip the Spectrum'),
                               amp=widgets.FloatRangeSlider(value=[0.00, 0.5], min=0, max=0.6, step=0.01, description='Amplitude Range', orientation='horizontal', readout=True),
                               sigma=widgets.FloatRangeSlider(value=[0, 0.3], min=0, max=2, step=0.01, description='Width Range', orientation='horizontal', readout=True),
                               gaussRange=widgets.IntRangeSlider(value=[1, 15], min=1, max=20, step=1, description='Range of Gaussians to Test', orientation='horizontal', readout=True),
                               convergence=widgets.FloatSlider(value=3.0, min=0., max=10., step=0.1, description=r'Convergence (1e-n)'),
                               maxIter=widgets.IntText(value=5000, step=1000, description='Maximum fitting iterations'),
                               baseLine=widgets.ToggleButtons(options=['Shift', 'Level', 'None'], value='None', description="Baseline Correction"),
                               preSmoothingSigma=widgets.IntSlider(value=0, min=0, max=20, description='Smoothing of Non-Deriv Space'),
                               smoothingSigma=widgets.IntSlider(value=0, min=0, max=20, description='Smoothing of Deriv Space'),
                               smoothingRaw=widgets.IntSlider(value=13, min=0, max=20, description='Smoothing of RAW Specrum'),
                               derivLevel=widgets.ToggleButtons(options=[0, 1, 2, 3], value=1, description="Derivative Level"),
                               ampCutoff=widgets.FloatRangeSlider(value=[0, 4], min=0, max=8, step=0.0001, description='Refit Amp Cutoff (1e-n)', orientation='horizontal', readout=True),
                               smoothingType=widgets.ToggleButtons(options=['Gaussian', 'Savitzky–Golay'], value='Savitzky–Golay', description="Smoothing Algorithm"),
                               savgolPoly=widgets.IntSlider(value=2, min=0, max=10, description='Savitzky–Golay Polynomial Order'),
                               savgolDeriv=widgets.IntSlider(value=0, min=0, max=10, description='Savitzky–Golay Derivative Level'),
                               save=widgets.ToggleButton(value=False, description="Save on Devconvolute"))

display(widgets.VBox([fc, fit_widg]))

display(HTML('''<style>
    .jupyter-widgets.lm-Widget {
        min-width: max-content;
        }
    .widget-label {
        min-width: 30ex;
        width: max-content;
        }
    .widget-button { min-width: max-content; }
    .jupyter-button { min-width: max-content; }
    .widget-hslider .slider-container {
        width: 50ex;
        max-width: 100%;
        }
</style>'''))

%matplotlib ipympl

# What the Knobs and Dials do
| Setting | Step it pertains to | What it does |
| :------ | :-----------------: | :----------- |
| Spectra Range (Manual Clipping) | 2 | Specifies the range of the spectrum to keep/analyse |
| Auto Clip the Spectrum | 4 | Turns on auto-clipping which splits the spectrum at the highest peak, and sets the limits at the lowest points eiother side of it | 
| Amplitude Range | 10 | Specifies the range of amplitudes for Gaussians that the least squares fitter is allowed to use |
| Width Range | 10 | Specifies the range of widths for Gaussians that the least squares fitter is allowed to use |
| Range of Gaussians to Test | 10 | Specifies the number of Gaussians to test to try and meet the convergence criteria |
| Convergence (1e-n)  | 10 | Specifies the residual that needss to be met for the fit to be considered successful |
| Maximum fitting iterations | 10 | Specifies how many iterations the least squares fitter can take |
| Baseline Correction | 3 | Shift lowers all the Y values so the lowest value sits on the baseline. Level does this as a linear interpolation between the lowest point on either side of the middle peak. **Only use level if you're sure it's what you need!** |
| Smoothing of Non-Deriv Space | 7 | This is the amount to smooth the spectrum before it's differentiated |
| Smoothing of Deriv Space | 8 | This is the amount to smooth the spectrum at each differentiation step |
| Smoothing of RAW Specrum | 5 | This is the amount to smooth the raw spectrum for refitting |
| Derivative Level | 8 | How many times to numerically differentiate the spectrum before fitting |
| Refit Amp Cutoff (1e-n) | 11 | Allows for removal of Gaussians with amplitudes that are either too small or too large |
| Smoothing Algorithm | 5, 7-8 | Whether to smooth the spectrum with a Gaussian kernel or SavGol filter |
| Savitzky–Golay Polynomial Order | 5, 7-8 | If using SavGol smoothing this is the polynomial order to fit (**Musst be larger than the smoothing level**) | 
| Savitzky–Golay Derivative Level | 5, 7-8 | If using SavGol smoothing this derivative space to smooth in | 
| Save on Devconvolute | 12 | Saves the fitting params, peaks and spectra in both deriv space and non-deriv space |